# Chapter 8: Data Wrangling - Join, Combine, and Reshape

In [7]:
import numpy as np
import pandas as pd
np.random.seed(42)

## 8.1 Hierarchical Indexing

*Hierarchical indexing* is an important feature of pandas that enables you to have multiple (two or more) index *levels* on an axis.

This is a prettified view of a Series with `MultiIndex` as its index. The "gaps" mean "use the label directly above."

In [19]:
data = pd.Series(np.random.randn(9),
                index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1   -0.234587
   2   -1.415371
   3   -0.420645
b  1   -0.342715
   3   -0.802277
c  1   -0.161286
   2    0.404051
d  2    1.886186
   3    0.174578
dtype: float64

In [20]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

`Partial` indexing is possible, enabling selection of subsets of data.

In [21]:
data['b']

1   -0.342715
3   -0.802277
dtype: float64

In [22]:
data['b': 'c']

b  1   -0.342715
   3   -0.802277
c  1   -0.161286
   2    0.404051
dtype: float64

In [23]:
data.loc[['b', 'd']]

b  1   -0.342715
   3   -0.802277
d  2    1.886186
   3    0.174578
dtype: float64

In [24]:
data.loc[:, 2]

# First selects all rows, then 2

a   -1.415371
c    0.404051
d    1.886186
dtype: float64

Hierarchical indexing plays an important role in reshaping data and group-based operations. 

For example, you could rearrange data into a DataFrame using `unstack()`.

In [25]:
data.unstack()

,1,2,3
a,-0.234587,-1.415371,-0.420645
b,-0.342715,NaN,-0.802277
c,-0.161286,0.404051,NaN
d,NaN,1.886186,0.174578


In [26]:
data.unstack().stack()

a  1   -0.234587
   2   -1.415371
   3   -0.420645
b  1   -0.342715
   3   -0.802277
c  1   -0.161286
   2    0.404051
d  2    1.886186
   3    0.174578
dtype: float64

With a DataFrame, either axis can have hierarchical index.

In [27]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                    index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                    columns=[['Ohio', 'Ohio', 'Colorado'],
                            ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [28]:
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

> Note: Be careful to distinguish the index names `'state'` and `'color'` from the row labels.

In [29]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

`MultiIndex` can be created by itself and then reused.

In [31]:
pd.MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'],
                            ['Green', 'Red', 'Green']],
                            names=['state', 'color'])

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           names=['state', 'color'])

### 8.1.1 Reordering and Sorting Levels

At times you will need to rearrange the order of the levels on an axis or sort the data by the values in one specific level.

`swaplevel` takes two level numbers or names and returns a new object with the levels interchanged.

In [32]:
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

`sort_index` sorts the data using only the values in a single level. It's also used to lexicographically sort at the indicated level.

In [33]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [34]:
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

### 8.1.2 Summary Statistics by Level

Many descriptive and summary statistics on DataFrame and Series have a `level` option in which you specify the level you want to aggregate by on a particular axis.

In [35]:
frame.sum(level='key2')

# Specifically in 'key2', sum up all the values in respective rows.

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [36]:
frame.sum(level='color', axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### 8.1.3 Indexing with a DataFrame's columns

It's not unusual to want to use one or more columns from a DataFrame as the row index.  
Or move the row index into a DataFrame column.

In [37]:
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                        'c': ['one', 'one', 'one', 'two', 'two',
                                'two', 'two'],
                        'd': [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


DataFrame's `set_index()` will create a new DataFrame using one or more of its columns as the index.

In [38]:
frame2 = frame.set_index(['c', 'd'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [39]:
frame.set_index(['c', 'd'], drop=False)

# By default columns are removed
# Leave them in with 'drop=False'

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

Use `reset_index()` to move the hierarchical index levels into the columns.

In [40]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## 8.2 Combining and Merging Datasets

### 8.2.1 Database-Style DataFrame Joins

### 8.2.2 Merging on Index

### 8.2.3 Concatenating Along an Axis

### 8.2.4 Combining Data with Overlap

## 8.3 Reshaping and Pivoting

### 8.3.1 Reshaping with Hierarchical Indexing

### 8.3.2 Pivoting "Long" to "Wide" Format

### 8.3.3 Pivoting "Wide" to "Long" Format

## 8.4 Conclusion